# Trading Signal Generation

## Setup

In [ ]:
import os
import sys

# Change working directory to the root of the project
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
os.chdir(project_root)

# Add 'src' to Python path
sys.path.append(os.path.join(project_root, 'src'))

In [ ]:
# -------------------- Core Libraries --------------------
import os
import re
import numpy as np
import pandas as pd

# -------------------- Text Preprocessing --------------------
from preprocessing import preprocess_text

# -------------------- Word Embedding --------------------
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from scipy.spatial.distance import pdist, squareform
from scipy.special import softmax

# -------------------- Machine Learning --------------------
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay
)
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# -------------------- Visualization --------------------
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# -------------------- Model Saving --------------------
import joblib

# -------------------- Pandas Display Options --------------------
pd.set_option('display.max_colwidth', None)


## Preprocessing

In [ ]:
file_path1 = "data/processed/labeled_january_data.csv"
with open(file_path1, "r", encoding="utf-8") as file:
    df_jan = pd.read_csv(file)

file_path2 = "data/processed/labeled_february_data.csv"
with open(file_path2, "r", encoding="utf-8") as file:
    df_feb = pd.read_csv(file)

file_path3 = "data/processed/labeled_march_data.csv"
with open(file_path3, "r", encoding="utf-8") as file:
    df_march = pd.read_csv(file)

In [ ]:
# Apply preprocessing to the dataset
df_jan['cleaned_article'] = df_jan['article'].apply(preprocess_text)
df_feb['cleaned_article'] = df_feb['article'].apply(preprocess_text)
df_march['cleaned_article'] = df_march['article'].apply(preprocess_text)

## Applying the Word2Vec approach
- custom word2vec model implemented by me

In [ ]:
# Prepare tokenized articles
tokenized_articles = df_jan['cleaned_article'].apply(str.split).tolist() + df_feb['cleaned_article'].apply(str.split).tolist() 
# Train Word2Vec
word2vec_model = Word2Vec(
    sentences=tokenized_articles,
    vector_size=300,  # Embedding dimensionality
    window=5,         # Context window size
    min_count=2,      # Minimum word frequency
    sg=1,             # Skip-gram method
    workers=4,        # Number of threads
    epochs=30         # Number of training iterations
)

word2vec_model.save("models/w2v_embeddings_experiment_v1/w2v.model")

In [ ]:
def vectorize_article(text, model):
    """
    Generate a document vector by averaging word embeddings.
    :param text: Preprocessed article as a string.
    :param model: Trained Word2Vec model.
    :return: Document vector (numpy array).
    """
    tokens = text.split()
    word_vectors = [model.wv[word] for word in tokens if word in model.wv]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(model.vector_size)

# Generate embeddings for both datasets
df_jan['embedding'] = df_jan['cleaned_article'].apply(lambda x: vectorize_article(x, word2vec_model))
df_feb['embedding'] = df_feb['cleaned_article'].apply(lambda x: vectorize_article(x, word2vec_model))
df_march['embedding'] = df_march['cleaned_article'].apply(lambda x: vectorize_article(x, word2vec_model))

In [ ]:
# Get most frequent words
most_frequent_words = word2vec_model.wv.index_to_key[:200]  # Top 200 words
word_vectors = np.array([word2vec_model.wv[word] for word in most_frequent_words])

# Reduce dimensionality using t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
word_embeddings_2d = tsne.fit_transform(word_vectors)

# Plot t-SNE visualization
plt.figure(figsize=(12, 8))
plt.scatter(word_embeddings_2d[:, 0], word_embeddings_2d[:, 1], alpha=0.7, edgecolors='k')
for i, word in enumerate(most_frequent_words):
    plt.annotate(word, (word_embeddings_2d[i, 0], word_embeddings_2d[i, 1]), fontsize=9)
plt.title('t-SNE Visualization of Word Embeddings')
plt.savefig("results/w2v_embeddings_experiment_v1/figures/word2vec_tsne_visualization.png")
plt.show()

## Visualising the distribution of the Target variable
- helps us to realize a class imbalance
- good to keep track of
- MOVE THIS INTO the PREPROCESSING file eventually

In [ ]:
sns.countplot(x='label', data=df_jan)
plt.title('Label Distribution in January Dataset')
plt.savefig("results/w2v_embeddings_experiment_v1/figures/label_distribution_january.png")
plt.show()

In [ ]:
sns.countplot(x='label', data=df_feb)
plt.title('Label Distribution in February Dataset')
plt.savefig("results/w2v_embeddings_experiment_v1/figures/label_distribution_february.png")
plt.show()

In [ ]:
sns.countplot(x='label', data=df_march)
plt.title('Label Distribution in March Dataset')
plt.savefig("results/w2v_embeddings_experiment_v1/figures/label_distribution_march.png")
plt.show()

In [ ]:
# Inspect class distribution
print("January class distribution:")
print(df_jan['label'].value_counts())

print("February class distribution:")
print(df_feb['label'].value_counts())

print("March class distribution (test set):")
print(df_march['label'].value_counts())

In [ ]:
def calculate_intra_class_distance(df, class_label):
    
    class_vectors = np.vstack(df[df['label'] == class_label]['embedding'].values)
    if len(class_vectors) <= 1:
        return 0  

    pairwise_distances = pdist(class_vectors, metric='euclidean')
    return np.mean(pairwise_distances)

def calculate_inter_class_distance(df, label_1, label_2):

    vectors_1 = np.vstack(df[df['label'] == label_1]['embedding'].values)
    vectors_2 = np.vstack(df[df['label'] == label_2]['embedding'].values)

    pairwise_distances = np.linalg.norm(vectors_1[:, None] - vectors_2, axis=2).flatten()
    return np.mean(pairwise_distances)

labels = [-1, 0, 1]
intra_class_distances = {label: calculate_intra_class_distance(df_jan, label) for label in labels}

inter_class_distances = {}
for i, label_1 in enumerate(labels):
    for label_2 in labels[i + 1:]:
        key = f"{label_1} vs {label_2}"
        inter_class_distances[key] = calculate_inter_class_distance(df_jan, label_1, label_2)

print("Intra-Class Distances:")
for label, dist in intra_class_distances.items():
    print(f"Label {label}: {dist:.4f}")

print("\nInter-Class Distances:")
for pair, dist in inter_class_distances.items():
    print(f"{pair}: {dist:.4f}")

## Multinomial Logistic Regression Model with Custom Word2Vec Model

Task: 
- Train on January, test on February
    - Train / test with 3 classes (+1, 0, -1), yielding a 3 x 3 confusion matrix
    - Train / test with 2 classes (+1, -1), yielding a 2 x 2 confusion matrix

In [157]:
# multiclass classification
# combine df_jan and df_feb for training
X_train_multi = np.vstack(np.concatenate((df_jan['embedding'].values, df_feb['embedding'].values)))
y_train_multi = np.concatenate((df_jan['label'], df_feb['label']))
X_test_multi = np.vstack(df_march['embedding'].values)
y_test_multi = df_march['label']

# binary classification
# filter out rows where label is 0 for binary classification
df_train_binary = pd.concat([df_jan, df_feb])
df_train_binary = df_train_binary[df_train_binary['label'] != 0]
df_march_binary = df_march[df_march['label'] != 0]

X_train_binary = np.vstack(df_train_binary['embedding'].values)
y_train_binary = df_train_binary['label']
X_test_binary = np.vstack(df_march_binary['embedding'].values)
y_test_binary = df_march_binary['label']

In [ ]:
# multinomial Logistic Regression (multi-class)
multi_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
multi_model.fit(X_train_multi, y_train_multi)
multi_preds = multi_model.predict(X_test_multi)

# binary Logistic regression (binary classification)
binary_model = LogisticRegression(max_iter=1000)
binary_model.fit(X_train_binary, y_train_binary)
binary_preds = binary_model.predict(X_test_binary)

In [ ]:

# Directory to save results
save_dir = "results/w2v_embeddings_experiment_v1/metrics"
os.makedirs(save_dir, exist_ok=True)

# Evaluate multi-class model
multi_class_report = classification_report(y_test_multi, multi_preds)
multi_class_cm = confusion_matrix(y_test_multi, multi_preds)

# Save multi-class metrics
with open(os.path.join(save_dir, "classification_report_multi.txt"), "w") as f:
    f.write("Multi-class Classification Report:\n")
    f.write(multi_class_report)

# Save multi-class confusion matrix
multi_class_cm_file = os.path.join(save_dir, "confusion_matrix_multi.png")
sns.heatmap(multi_class_cm, annot=True, fmt='d', cmap='Blues', xticklabels=["-1", "0", "1"], yticklabels=["-1", "0", "1"])
plt.title("Multi-class Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.savefig(multi_class_cm_file)
plt.close()

In [160]:
# Evaluate binary model
binary_class_report = classification_report(y_test_binary, binary_preds)
binary_class_cm = confusion_matrix(y_test_binary, binary_preds)

# Save binary metrics
with open(os.path.join(save_dir, "classification_report_binary.txt"), "w") as f:
    f.write("Binary Classification Report:\n")
    f.write(binary_class_report)

# Save binary confusion matrix
binary_class_cm_file = os.path.join(save_dir, "confusion_matrix_binary.png")
sns.heatmap(binary_class_cm, annot=True, fmt='d', cmap='Greens', xticklabels=["-1", "1"], yticklabels=["-1", "1"])
plt.title("Binary Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.savefig(binary_class_cm_file)
plt.close()

In [ ]:
joblib.dump(multi_model, 'models/w2v_embeddings_experiment_v1/multi_class_model.pkl')
joblib.dump(binary_model, 'models/w2v_embeddings_experiment_v1/binary_model.pkl')